# CNC Mill Tool Wear
- https://www.kaggle.com/shasun/tool-wear-detection-in-cnc-mill
- Predict "Machining_Process"

In [105]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

In [106]:
saveFolder = 'Data_CNC_processed'
filename = 'pred_machining_process'
compression = 'gzip'
df = pd.read_pickle(f'{saveFolder}/{filename}.pkl',compression=compression)
df.head(3)

,X1_ActualPosition,X1_ActualVelocity,X1_ActualAcceleration,X1_CommandPosition,X1_CommandVelocity,X1_CommandAcceleration,X1_CurrentFeedback,X1_DCBusVoltage,X1_OutputCurrent,X1_OutputVoltage,...,S1_OutputCurrent,S1_OutputVoltage,S1_OutputPower,S1_SystemInertia,M1_CURRENT_PROGRAM_NUMBER,M1_sequence_number,M1_CURRENT_FEEDRATE,Machining_Process,feedrate,clamp_pressure
0,198.0,0.0,0.00,198.0,0.0,0.000000,0.18,0.0207,329.0,2.77,...,329.0,0.0,6.960000e-07,12.0,1.0,0.0,50.0,7,6,4.0
1,198.0,-10.8,-350.00,198.0,-13.6,-358.000000,-10.90,0.1860,328.0,23.30,...,328.0,0.0,-5.270000e-07,12.0,1.0,4.0,50.0,7,6,4.0
2,196.0,-17.8,-6.25,196.0,-17.9,-0.000095,-8.59,0.1400,328.0,30.60,...,328.0,0.0,9.100000e-07,12.0,1.0,7.0,50.0,7,6,4.0


In [107]:
colsX = [item for item in df.columns if item not in 'Machining_Process'] 
X = df[colsX].values
y = df['Machining_Process'].values

### Result
- Liblinear yield result with convergence.
- Sag and Saga yield convergence with some C.

In [108]:
paramSetLgfgs = {
    "ex1": {"solver": "lbfgs", "C": 0.0001, "max_iter": 100},
    "ex2": {"solver": "lbfgs", "C": 0.01, "max_iter": 100},
    "ex3": {"solver": "lbfgs", "C": 1, "max_iter": 100},
    "ex4": {"solver": "lbfgs", "C": 100, "max_iter": 100},
    "ex5": {"solver": "lbfgs", "C": 10000, "max_iter": 100},
    }

paramSetSag = {
    "ex1": {"solver": "sag", "C": 0.0001, "max_iter": 100},
    "ex2": {"solver": "sag", "C": 0.01, "max_iter": 100},
    "ex3": {"solver": "sag", "C": 1, "max_iter": 100},
    "ex4": {"solver": "sag", "C": 100, "max_iter": 100},
    "ex5": {"solver": "sag", "C": 10000, "max_iter": 100},
    }

paramSetLibLinear = {
    "ex1": {"solver": "liblinear", "C": 0.0001, "max_iter": 100},
    "ex2": {"solver": "liblinear", "C": 0.01, "max_iter": 100},
    "ex3": {"solver": "liblinear", "C": 1, "max_iter": 100},
    "ex4": {"solver": "liblinear", "C": 100, "max_iter": 100},
    "ex5": {"solver": "liblinear", "C": 10000, "max_iter": 100},
    }

paramSetSaga = {
    "ex1": {"solver": "saga", "C": 0.0001, "max_iter": 100},
    "ex2": {"solver": "saga", "C": 0.01, "max_iter": 100},
    "ex3": {"solver": "saga", "C": 1, "max_iter": 100},
    "ex4": {"solver": "saga", "C": 100, "max_iter": 100},
    "ex5": {"solver": "saga", "C": 10000, "max_iter": 100}
    }

In [109]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=1, stratify=y
)

In [110]:
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

In [111]:
#paramSubSet = {k:v for k,v in paramSet.items() if k in ['ex7', 'ex8', 'ex9', 'ex10']}
paramSubSet = paramSetLibLinear

In [112]:
for paramName, paramValue in paramSubSet.items():
    lr = LogisticRegression(
        random_state=1,
        verbose=0,
        solver=paramValue["solver"],
        C=paramValue["C"],
        max_iter=paramValue["max_iter"],
    )

    # Training
    lr.fit(X_train_std, y_train)

    # Prediction
    y_pred = lr.predict(X_test_std)

    # Misclassification from the test samples
    sumMiss = (y_test != y_pred).sum()

    # Accuracy score from the test samples
    accuracyScore = accuracy_score(y_test, y_pred)

    print(f"Parameters: {paramValue}")
    print(f"Misclassified examples: {sumMiss}")
    print(f"Accuracy score: {accuracyScore}")
    print(f"Norm of W: {np.linalg.norm(lr.coef_)}")
    print(f"--------------------------------------------------")

Parameters: {'solver': 'liblinear', 'C': 0.0001, 'max_iter': 100}
Misclassified examples: 6371
Accuracy score: 0.16016345900342738
Norm of W: 0.14341522663502546
--------------------------------------------------
Parameters: {'solver': 'liblinear', 'C': 0.01, 'max_iter': 100}
Misclassified examples: 6148
Accuracy score: 0.18955971526496176
Norm of W: 1.6376864165442313
--------------------------------------------------
Parameters: {'solver': 'liblinear', 'C': 1, 'max_iter': 100}
Misclassified examples: 6122
Accuracy score: 0.19298708146585816
Norm of W: 5.819687866239522
--------------------------------------------------
Parameters: {'solver': 'liblinear', 'C': 100, 'max_iter': 100}
Misclassified examples: 6132
Accuracy score: 0.19166886369628264
Norm of W: 24.260189152329986
--------------------------------------------------
Parameters: {'solver': 'liblinear', 'C': 10000, 'max_iter': 100}
Misclassified examples: 6130
Accuracy score: 0.19193250725019773
Norm of W: 115.49566869997969
--